In [1]:
!pip install tensorflow-gpu==1.13.1

     |████████████████████████████████| 345.2MB 112kB/s 
     |████████████████████████████████| 3.2MB 23.9MB/s 
     |████████████████████████████████| 368kB 61.2MB/s 
ERROR: tensorflow 1.15.0rc3 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 1.13.1 which is incompatible.
ERROR: tensorflow 1.15.0rc3 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 1.13.0 which is incompatible.
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1


In [2]:

!pip install funcy

In [3]:

# from data_set                  import load_data
from funcy                     import concat, identity, juxt, partial, rcompose, repeat, repeatedly, take
from keras.callbacks           import LearningRateScheduler
from keras.layers              import Activation, Add, BatchNormalization, Conv2D, Dense, GlobalAveragePooling2D, Input
from keras.models              import Model, save_model
from keras.optimizers          import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers        import l2
from keras.utils               import plot_model
from operator                  import getitem
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets
from keras.layers.advanced_activations import PReLU
from scipy import misc
import numpy
import numpy.random


import numpy as np
import sklearn.metrics as metrics
from keras import backend as K
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import os
import distutils
import tensorflow as tf
os.environ["KERAS_BACKEND"] = "tensorflow"

from keras.engine import Layer, InputSpec
from keras import initializers
from keras import regularizers
from keras import constraints
from keras import backend as K

from keras.utils.generic_utils import get_custom_objects
import  tensorflow as tf
# aaa=tf.keras.layers.Input((1,1,1))
# if distutils.version.LooseVersion(tf.__version__) < '1.14':
#     raise Exception('This notebook is compatible with TensorFlow 1.14 or higher, for TensorFlow 1.13 or lower please use the previous version at https://github.com/tensorflow/tpu/blob/r1.13/tools/colab/fashion_mnist.ipynb')



class SwitchableNormalization(Layer):
    """Switchable normalization layer

    # Arguments
        axis: Integer, the axis that should be normalized
            (typically the features axis).
            For instance, after a `Conv2D` layer with
            `data_format="channels_first"`,
            set `axis=1` in `BatchNormalization`.
        momentum: Momentum for the moving mean and the moving variance.
        epsilon: Small float added to variance to avoid dividing by zero.
        center: If True, add offset of `beta` to normalized tensor.
            If False, `beta` is ignored.
        scale: If True, multiply by `gamma`.
            If False, `gamma` is not used.
            When the next layer is linear (also e.g. `nn.relu`),
            this can be disabled since the scaling
            will be done by the next layer.
        mean_weight_initializer: Initializer for mean weight
        variance_weight_initializer: Initializer for variance weight
        beta_initializer: Initializer for the beta weight.
        gamma_initializer: Initializer for the gamma weight.
        beta_regularizer: Optional regularizer for the beta weight.
        gamma_regularizer: Optional regularizer for the gamma weight.
        beta_constraint: Optional constraint for the beta weight.
        gamma_constraint: Optional constraint for the gamma weight.

    # Input shape
        Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a model.

    # Output shape
        Same shape as input.

    # References
        - [Differentiable Learning-to-Normalize via Switchable Normalization](https://arxiv.org/abs/1806.10779)
        - [SN by pytorch](https://github.com/switchablenorms/Switchable-Normalization/blob/master/models/switchable_norm.py)
    """

    def __init__(self,
                 axis=-1,
                 momentum=0.99,
                 epsilon=1e-5,
                 center=True,
                 scale=True,
                 mean_weight_initializer='ones',
                 variance_weight_initializer='ones',
                 beta_initializer='zeros',
                 gamma_initializer='ones',
                 moving_mean_initializer='zeros',
                 moving_variance_initializer='ones',
                 beta_regularizer=None,
                 gamma_regularizer=None,
                 beta_constraint=None,
                 gamma_constraint=None,
                 **kwargs):
        super(SwitchableNormalization, self).__init__(**kwargs)
        self.supports_masking = True
        self.axis = axis
        self.momentum = momentum
        self.epsilon = epsilon
        self.center = center
        self.scale = scale
        self.mean_weight_initializer = initializers.get(mean_weight_initializer)
        self.variance_weight_initializer = initializers.get(variance_weight_initializer)
        self.beta_initializer = initializers.get(beta_initializer)
        self.gamma_initializer = initializers.get(gamma_initializer)
        self.moving_mean_initializer = initializers.get(moving_mean_initializer)
        self.moving_variance_initializer = initializers.get(moving_variance_initializer)
        self.beta_regularizer = regularizers.get(beta_regularizer)
        self.gamma_regularizer = regularizers.get(gamma_regularizer)
        self.beta_constraint = constraints.get(beta_constraint)
        self.gamma_constraint = constraints.get(gamma_constraint)

    def build(self, input_shape):
        dim = input_shape[self.axis]

        if dim is None:
            raise ValueError('Axis ' + str(self.axis) + ' of '
                                                        'input tensor should have a defined dimension '
                                                        'but the layer received an input with shape ' +
                             str(input_shape) + '.')

        self.input_spec = InputSpec(ndim=len(input_shape),
                                    axes={self.axis: dim})

        # add mean/variance weight
        self.mean_weight = self.add_weight(shape=(3,),
                                           name="mean_weight",
                                           initializer=self.mean_weight_initializer)
        self.variance_weight = self.add_weight(shape=(3,),
                                               name="variance_weight",
                                               initializer=self.variance_weight_initializer)

        # add gamma/beta weight
        shape = (dim,)
        if self.scale:
            self.gamma = self.add_weight(shape=shape,
                                         name='gamma',
                                         initializer=self.gamma_initializer,
                                         regularizer=self.gamma_regularizer,
                                         constraint=self.gamma_constraint)
        else:
            self.gamma = None
        if self.center:
            self.beta = self.add_weight(shape=shape,
                                        name='beta',
                                        initializer=self.beta_initializer,
                                        regularizer=self.beta_regularizer,
                                        constraint=self.beta_constraint)
        else:
            self.beta = None

        self.moving_mean = self.add_weight(
            shape=shape,
            name="moving_mean",
            initializer=self.moving_mean_initializer,
            trainable=False)
        self.moving_variance = self.add_weight(
            shape=shape,
            name="moving_variance",
            initializer=self.moving_variance_initializer,
            trainable=False)

        self.built = True

    def call(self, inputs, training=None):
        input_shape = K.int_shape(inputs)
        # Prepare broadcasting shape.
        ndim = len(input_shape)
        broadcast_shape = [1] * len(input_shape)
        broadcast_shape[self.axis] = input_shape[self.axis]

        # mean/variance of instance_normalization
        reduction_axes_in = list(range(len(input_shape)))
        del reduction_axes_in[self.axis]
        del reduction_axes_in[0]
        mean_in = K.mean(inputs, axis=reduction_axes_in, keepdims=True)
        variance_in = K.var(inputs, axis=reduction_axes_in, keepdims=True)

        # mean/variance of layer_normalization
        reduction_axes_ln = list(range(len(input_shape)))
        del reduction_axes_ln[0]
        mean_ln = K.mean(inputs, axis=reduction_axes_ln, keepdims=True)
        variance_ln = K.var(inputs, axis=reduction_axes_ln, keepdims=True)

        # mean/variance of batch_normalization
        reduction_axes_bn = list(range(len(input_shape)))
        del reduction_axes_bn[self.axis]

        def normed_training():
            mean_bn = K.mean(inputs, axis=reduction_axes_bn, keepdims=True)
            variance_bn = K.var(inputs, axis=reduction_axes_bn, keepdims=True)
            mean = [mean_in, mean_ln, mean_bn]
            variance = [variance_in, variance_ln, variance_bn]

            # If the learning is either dynamic, or set to training:
            self.add_update([K.moving_average_update(self.moving_mean,
                                                     K.reshape(mean_bn, (input_shape[self.axis],)),
                                                     self.momentum),
                             K.moving_average_update(self.moving_variance,
                                                     K.reshape(variance_bn, (input_shape[self.axis],)),
                                                     self.momentum)],
                            inputs)
            return norm(mean, variance)

        def normalize_inference():
            mean_bn = self.moving_mean
            variance_bn = self.moving_variance
            mean = [mean_in, mean_ln, mean_bn]
            variance = [variance_in, variance_ln, variance_bn]
            return norm(mean, variance)

        def norm(mean, variance):
            mean_weight = K.softmax(self.mean_weight)
            variance_weight = K.softmax(self.variance_weight)
            norm_mean = mean_weight[0] * mean[0] + mean_weight[1] * mean[1] + mean_weight[2] * mean[2]
            norm_variance = variance_weight[0] * variance[0] + variance_weight[1] * variance[1] + variance_weight[2] * \
                            variance[2]
            normd = (inputs - norm_mean) / (K.sqrt(norm_variance + self.epsilon))
            if self.scale:
                broadcast_gamma = K.reshape(self.gamma, broadcast_shape)
                normd = normd * broadcast_gamma
            if self.center:
                broadcast_beta = K.reshape(self.beta, broadcast_shape)
                normd = normd + broadcast_beta
            return normd

        if training in {0, False}:
            return normalize_inference

        # Pick the normalized form corresponding to the training phase.
        return K.in_train_phase(normed_training,
                                normalize_inference,
                                training=training)

    def get_config(self):
        config = {
            'axis': self.axis,
            'momentum': self.momentum,
            'epsilon': self.epsilon,
            'center': self.center,
            'scale': self.scale,
            'mean_weight_initializer': self.mean_weight_initializer,
            'variance_weight_initializer': self.variance_weight_initializer,
            'beta_initializer': initializers.serialize(self.beta_initializer),
            'gamma_initializer': initializers.serialize(self.gamma_initializer),
            'moving_mean_initializer': initializers.serialize(self.moving_mean_initializer),
            'moving_variance_initializer': initializers.serialize(self.moving_variance_initializer),
            'beta_regularizer': regularizers.serialize(self.beta_regularizer),
            'gamma_regularizer': regularizers.serialize(self.gamma_regularizer),
            'beta_constraint': constraints.serialize(self.beta_constraint),
            'gamma_constraint': constraints.serialize(self.gamma_constraint)
        }
        base_config = super(SwitchableNormalization, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
        return input_shape

def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=1, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser


def random_crop_image(image):
      height, width = image.shape[:2]
      random_array = numpy.random.random(size=4)
      w = int((width*0.5)*(1+random_array[0]*0.5))
      h = int((height*0.5)*(1+random_array[1]*0.5))
      x = int(random_array[2]*(width-w))
      y = int(random_array[3]*(height-h))

      image_crop = image[y:h+y,x:w+x,0:3]
      image_crop = misc.imresize(image_crop,image.shape)
      return image_crop

import keras
from keras.models import Model
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import Input, Add, GlobalAveragePooling2D, Dropout
from keras import regularizers

weight_decay = 5e-4


def conv3x3(input, out_planes, stride=1):
    """3x3 convolution with padding"""
    return tf.keras.layers.Conv2D(out_planes, kernel_size=3, strides=stride,
                    padding='same', use_bias=False, kernel_initializer='he_normal',
                    kernel_regularizer=regularizers.l2(weight_decay))(input)


def conv1x1(input, out_planes, stride=1):
    """1x1 convolution"""
    return tf.keras.layers.Conv2D(out_planes, kernel_size=1, strides=stride,
                    padding='same', use_bias=False, kernel_initializer='he_normal',
                    kernel_regularizer=regularizers.l2(weight_decay))(input)


def BasicBlock(input, planes, dropout, stride=1):
    inplanes = input.shape[3]

    out = tf.keras.layers.BatchNormalization()(input)
    # out = SwitchableNormalization()(input)
    out = tf.keras.layers.Activation('elu')(out)
    out = conv3x3(out, planes, stride)
    out = tf.keras.layers.BatchNormalization()(out)
    # out = SwitchableNormalization()(out)
    out = tf.keras.layers.Activation('elu')(out)
    out = tf.keras.layers.Dropout(dropout)(out)
    out = conv3x3(out, planes)

    if stride != 1 or inplanes != planes:
        shortcut = conv1x1(input, planes, stride)
    else:
        shortcut = out

    out = tf.keras.layers.Add()([out, shortcut])

    return out


def WideResNet(depth=40, width=10,num_classes=10, dropout=0.3):
    layer = (depth - 4) // 6

    input = tf.keras.layers.Input(shape=(28, 28, 1))

    x = conv3x3(input, 16)
    for _ in range(layer):
        x = BasicBlock(x, 16*width, dropout)
    x = BasicBlock(x, 32*width, dropout, 2)
    for _ in range(layer-1):
        x = BasicBlock(x, 32*width, dropout)
    x = BasicBlock(x, 64*width, dropout, 2)
    for _ in range(layer-1):
        x = BasicBlock(x, 64*width, dropout)

    x = tf.keras.layers.BatchNormalization()(x)
    # x = SwitchableNormalization()(x)
    x = tf.keras.layers.Activation('elu')(x)
    print(x.shape)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
#     x = tf.keras.layers.Conv2D(64, kernel_size=3, strides=2,
#                     padding='valid', use_bias=False, kernel_initializer='he_normal',
#                     kernel_regularizer=regularizers.l2(weight_decay))(x)
#     x = tf.keras.layers.Conv2D(64, kernel_size=3, strides=2,
#                     padding='valid', use_bias=False, kernel_initializer='he_normal',
#                     kernel_regularizer=regularizers.l2(weight_decay))(x)
#     x =tf.keras.layers.Flatten()(x)
    # x=tf.reshape(x,shape=(-1,64))
    output = tf.keras.layers.Dense(num_classes, activation='softmax', kernel_regularizer=regularizers.l2(weight_decay))(x)

    model = tf.keras.Model(input, output)
    model.summary()

    return model



Using TensorFlow backend.


In [4]:
import os
from tensorflow.contrib.tpu.python.tpu import keras_support
from keras.utils import np_utils
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import load_model
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
print(np.min(x_train),np.max(x_train))
# add empty color dimension
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
x_train = (np.float32(x_train)/255.-0.1307)/0.3081
x_test = (np.float32(x_test)/255.-0.1307)/0.3081
y_train = np_utils.to_categorical(y_train,10)
y_test = np_utils.to_categorical(y_test,10)
print(np.min(x_train),np.max(x_train))
# resolver = tf.contrib.cluster_resolver.TPUClusterResolver('grpc://' + os.environ['COLAB_TPU_ADDR'])
# tf.contrib.distribute.initialize_tpu_system(resolver)
# strategy = tf.contrib.distribute.TPUStrategy(resolver)

# with strategy.scope():
#   model = create_model()
tpu_model =WideResNet()
tpu_model.compile(
    optimizer=    tf.keras.optimizers.SGD(lr=0.01,
             momentum=0.9,),
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy'])

tpu_grpc_url = "grpc://"+os.environ["COLAB_TPU_ADDR"]
tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(tpu_grpc_url)
strategy = keras_support.TPUDistributionStrategy(tpu_cluster_resolver)


tpu_model = tf.contrib.tpu.keras_to_tpu_model(tpu_model, strategy=strategy)




train_data  = tf.keras.preprocessing.image.ImageDataGenerator(featurewise_center=True,
                                     featurewise_std_normalization=True,
                                     # preprocessing_function=random_crop_image,
                                     preprocessing_function=get_random_eraser(v_l=0, v_h=1),
#                                      width_shift_range=0.125,
#                                      height_shift_range=0.125,
                                     horizontal_flip=True)
validation_data = tf.keras.preprocessing.image.ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)

for data in (train_data, validation_data):
    data.fit(x_train)  

batch_size = 400
epoch_size = 100000


weights_file = "./wrn_tpu.h5"

# if os.path.exists(weights_file):
#     tpu_model.load_weights(weights_file)
#     tpu_model = load_model(weights_file)
#     print("Model loaded 1.")
#     tpu_model.save(weights_file)

if os.path.exists(weights_file):
#     tpu_model.load_weights(weights_file)
    tpu_model = load_model(weights_file)
    print("Model loaded.")
    tpu_model = tf.contrib.tpu.keras_to_tpu_model(tpu_model, strategy=strategy)
    print('tpu schedule')

# lr_reducer = tf.keras.callbacks.LearningRateScheduler(monitor='val_loss', factor=np.sqrt(0.1),
#                                cooldown=0, patience=10, min_lr=1e-6)

model_checkpoint = keras.callbacks.ModelCheckpoint(weights_file, monitor="val_categorical_accuracy", save_best_only=True,
                                   save_weights_only=False, mode='auto')

callbacks = [tf.keras.callbacks.LearningRateScheduler(partial(getitem, tuple(take(epoch_size, concat(repeat(0.1, 60), repeat(0.02, 60), repeat(0.004, 40), repeat(0.0008)))))), model_checkpoint]

results = tpu_model.fit_generator(train_data.flow(x_train, y_train, batch_size=batch_size),
                              steps_per_epoch=60000// batch_size,
                              epochs=epoch_size,
                              callbacks=callbacks,
                              validation_data=validation_data.flow(x_test, y_test, batch_size=batch_size),
                              validation_steps=10000 // batch_size)

# results = model.fit(x_train.astype(np.float32), y_train.astype(np.float32),
#                               steps_per_epoch=int(60000// batch_size),
#                               epochs=epoch_size,
#                               callbacks=callbacks,
#                               validation_data=(x_test.astype(np.float32), y_test.astype(np.float32)),
#                               validation_steps=int(10000 // batch_size))

# with open('./results/history.pickle', 'wb') as f:
#     pickle.dump(results.history, f)

model.save_weights('./wrn_tpu.h5', overwrite=True)
  
  


# model.save_weights('./fashion_mnist.h5', overwrite=True)

4423680/4422102 [==============================] - 0s 0us/step
0 255
-0.42421296 2.8214867
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
(?, 7, 7, 640)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 28, 28, 16)   144         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_v1 (BatchNo (None, 28, 28, 16)   64          conv2d[0][0]                     
______________

KeyboardInterrupt: ignored